# SCRAPE DATA

In [329]:
# import libraries
# https://medium.com/codex/how-i-web-scraped-my-custom-google-maps-list-into-a-csv-file-eb1172a85bf4

import time
import timeit
from bs4 import BeautifulSoup

from selenium import webdriver

# initiate driver
driver = webdriver.Chrome()

# open url (link to google maps list)
url = 'https://www.google.com/maps/@33.5907877,130.3814759,14z/data=!3m1!4b1!4m3!11m2!2sVJhNXQIN1FHaiTf11xl6IV8gEwL_VQ!3e3?entry=ttu'
driver.get(url)

In [7]:
# scroll to the bottom of the list

start = timeit.default_timer()

# identify scrolling element first (sidebar)
scrolling_element_xpath = '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]'
scrolling_element = driver.find_element("xpath", scrolling_element_xpath)

# use height of element to determine if need to scroll
last_height = driver.execute_script('return arguments[0].scrollHeight', scrolling_element)
print(last_height)

SCROLL_PAUSE_TIME = 2.0 # pause before next scroll, to let page load
t = 0 # number of times we have scrolled

# loop the scrolling until cannot scroll anymore
while True:
    print(t)
    # scroll down to bottom of whatever is currently loaded
    driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', scrolling_element)
    t = t+1

    # wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # check if more scrolling required
    new_height = driver.execute_script('return arguments[0].scrollHeight', scrolling_element)
    print(new_height)
    if new_height == last_height:
        break
    last_height = new_height

stop =timeit.default_timer()

print('Time taken: ', stop - start)

8466
0
8466
Time taken:  2.0963863059998857


In [24]:
# get the page html, inspect code to find appropriate selector
inq_soup = BeautifulSoup(driver.page_source, 'html.parser')
print(inq_soup.prettify())

# select each of the saved locations
# selector is taken by clicking 'copy selector' in inspect
selector = '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ussYcc > div'
items = inq_soup.select(selector)

items_cleaned = items[:-1]

print(len(items_cleaned))

<html itemscope="" itemtype="http://schema.org/Place" jstcache="0" lang="ko" style='font-family: Roboto, "Noto Sans KR", Arial, sans-serif;'>
 <head>
  <link as="script" href="/maps/_/js/k=maps.m.ko.0W5drv23Vm8.2019.O/m=sc2,per,mo,lp,ti,ds,stx,dwi,enr,b/am=BgAjAAI/rt=j/d=1/rs=ACT90oFtP9wptChqsyqWXR3snr-VQwxqIA?wli=m.2fBy-DKK0I0.loadSv.O%3A%3Bm.AzvEr4SCTgs.mapcore.O%3A%3B" nonce="" rel="preload" type="application/javascript"/>
  <link href="/maps/preview/opensearch.xml?hl=ko" rel="search" title="Google 지도" type="application/opensearchdescription+xml"/>
  <title>
   후쿠오카 식사류 맛집 - Google 지도
  </title>
  <meta content="Google 지도에서 지역정보를 검색하고 지도를 살펴보거나 운전경로 정보를 검색합니다." name="Description"/>
  <meta content="Anm+hhtuh7NJguqSnXHEAIqqMaV+GXCks8WYXHJKF7l6AeYMj+wO+fi9OdDqFnJTg9t0492DykVxx4jpvFbxnA8AAABseyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IlByaXZhY3lTYW5kYm94QWRzQVBJcyIsImV4cGlyeSI6MTY5NTE2Nzk5OSwiaXNTdWJkb21haW4iOnRydWV9" http-equiv="origin-trial"/>
  <meta content="initia

In [51]:
print(items_cleaned[3])

<div class="m6QErb" style=""><div class="BsJqK xgHk6"><div class="X7m7Ff OyjIsf"></div><button class="SMP2wb fHEb6e" jsaction="pane.wfvdle375;mouseover:pane.wfvdle375;mouseout:pane.wfvdle375;focus:pane.wfvdle375;blur:pane.wfvdle375" jslog="39539; track:click;metadata:WyIwYWhVS0V3alNqdW1XOU1xRUF4WDFrMVlCSFdfR0Foa1E4QmNJcGdFb0FBIl0="><div class="GYMy9d UocPK" jsaction="pane.wfvdle376" jslog="25178;track:click;metadata:WyIwYWhVS0V3alNqdW1XOU1xRUF4WDFrMVlCSFdfR0Foa1F6Q2NJeXdFb0FBIl0="><img alt="" aria-hidden="true" class="WkIe8" crossorigin="" src="https://lh5.googleusercontent.com/p/AF1QipPnPxRyeodobHJLMPCQ6Vp_O44c2uvNECcCto-k=w122-h92-k-no"/></div><div class="H1bDYe fontBodyMedium" style='font-family: Roboto, "Noto Sans KR", Arial, sans-serif;'><div class="fontHeadlineSmall rZF81c" style='font-family: "Google Sans", Roboto, "Noto Sans KR", Arial, sans-serif;'>사케토 교자토 시타고코로</div><div class="yfRytc xua1Rc"><div class="IIrLbb"><div class="Rv8Tad"><span class="LYDYnf"><span class="e4rVHe ALW

In [70]:
import re
# use html selectors to extract info from each item saved in your list

def extract_text(html, section, class_, find_span=False):
    elements = html.find_all(section, class_=class_)
    if find_span:
        text = elements[1]
        text = re.sub(r'[^가-힣\s]', '', text.text)
        return text.strip()
    return elements[0].text.strip()

## places
places = [extract_text(i, 'div', 'fontHeadlineSmall rZF81c') for i in items_cleaned]

## ratings
ratings = [extract_text(i, 'span', 'MW4etd') for i in items_cleaned]

## category
categories = [extract_text(i, 'div', 'IIrLbb', find_span=True) for i in items_cleaned]

# put into df
import pandas as pd
import numpy

data = {'place': places,
        'category': categories,
        'rating': ratings}

df = pd.DataFrame(data)
df.head(10)

,place,category,rating
0,닌교초이마한 스키야끼,샤부샤부,4.4
1,우오츄,일식당 및 일정식집,4.4
2,야키니쿠 바쿠로 하카타점,야키니쿠,4.4
3,사케토 교자토 시타고코로,이자카야,4.4
4,미스즈안 우동 소바,소바 전문점,4.0
5,갓포 아지미,일식,4.6
6,하카타 모츠나베 마에다야 총본점,일식 내장 냄비 요리 전문점,4.6
7,사바타로,아침식사,4.6
8,오뎅니이노,어묵 전문식당,4.4
9,미트 텐진 호르몬 이마이즈미,야키니쿠,4.3


In [82]:
# retrieve url

def click_element(i):
    print('clicking item')
    # select item using xpath
    xpath = f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[{str(i)}]'
    element = driver.find_element("xpath", xpath)

    # click into listing
    element.click()
    time.sleep(5.0)

def back_to_list():
    print('going back to list')
    #driver.execute_script('window.history.go(-1)')
    # select item using xpath
    xpath = f'//*[@id="omnibox-singlebox"]/div/div[1]/button/span'
    element = driver.find_element("xpath", xpath)

    # click into listing
    element.click()
    time.sleep(5.0)

# function to load listing to be visible and selectable on sidebar
def load_listings(i):
    # must scroll for every 20 listings
    x = (i-1)//20
    print('loading scrollbar...', x, ' times')

    scrolling_element_xpath = '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]'
    scrolling_element = driver.find_element("xpath", scrolling_element_xpath)

    for j in range(0, x):
        driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', scrolling_element)
        time.sleep(2.0)

In [83]:
from selenium.common.exceptions import NoSuchElementException

start = timeit.default_timer()
driver.get(url)
time.sleep(2.0)

urls = []
errors = 0

for i in range(1, len(df)+1):
    print(i)
    try:
        if ((i-1)//20) > 0:
            load_listings(i)
        click_element(i)

    except NoSuchElementException:
        errors += 1
        print('running NoSuchElementException...sleeping 5 sec')
        time.sleep(2.0)
        back_to_list()
        if ((i-1)//20) > 0:
            load_listings(i)
        click_element(i)
        print('end of exception')

    # append url
    element_url = driver.current_url
    urls.append(element_url)

    # go back to list and try next item
    back_to_list()

stop = timeit.default_timer()

print('time taken: ', stop - start)
print('errors: ', errors)

1
clicking item
going back to list
2
clicking item
going back to list
3
clicking item
going back to list
4
clicking item
going back to list
5
clicking item
going back to list
6
clicking item
going back to list
7
clicking item
going back to list
8
clicking item
going back to list
9
clicking item
going back to list
10
clicking item
going back to list
11
clicking item
going back to list
12
clicking item
going back to list
13
clicking item
going back to list
14
clicking item
going back to list
15
clicking item
going back to list
16
clicking item
going back to list
17
clicking item
going back to list
18
clicking item
going back to list
19
clicking item
going back to list
20
clicking item
going back to list
21
loading scrollbar... 1  times
clicking item
going back to list
22
loading scrollbar... 1  times
clicking item
going back to list
23
loading scrollbar... 1  times
clicking item
going back to list
24
loading scrollbar... 1  times
clicking item
going back to list
25
loading scrollbar... 1

In [87]:
df['urls'] = urls

df.head(10)

,place,category,rating,urls
0,닌교초이마한 스키야끼,샤부샤부,4.4,https://www.google.com/maps/place/%EB%8B%8C%EA...
1,우오츄,일식당 및 일정식집,4.4,https://www.google.com/maps/place/%EC%9A%B0%EC...
2,야키니쿠 바쿠로 하카타점,야키니쿠,4.4,https://www.google.com/maps/place/%EC%95%BC%ED...
3,사케토 교자토 시타고코로,이자카야,4.4,https://www.google.com/maps/place/%EC%82%AC%EC...
4,미스즈안 우동 소바,소바 전문점,4.0,https://www.google.com/maps/place/%EB%AF%B8%EC...
5,갓포 아지미,일식,4.6,https://www.google.com/maps/place/%EA%B0%93%ED...
6,하카타 모츠나베 마에다야 총본점,일식 내장 냄비 요리 전문점,4.6,https://www.google.com/maps/place/%ED%95%98%EC...
7,사바타로,아침식사,4.6,https://www.google.com/maps/place/%EC%82%AC%EB...
8,오뎅니이노,어묵 전문식당,4.4,https://www.google.com/maps/place/%EC%98%A4%EB...
9,미트 텐진 호르몬 이마이즈미,야키니쿠,4.3,https://www.google.com/maps/place/%EB%AF%B8%ED...


In [299]:
# scroll to the bottom of the list

def scroll_down():
    start = timeit.default_timer()

    # identify scrolling element first (sidebar)
    scrolling_element_xpath = '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]'
    scrolling_element = driver.find_element("xpath", scrolling_element_xpath)

    # use height of element to determine if need to scroll
    last_height = driver.execute_script('return arguments[0].scrollHeight', scrolling_element)
    print(last_height)

    SCROLL_PAUSE_TIME = 2.0 # pause before next scroll, to let page load
    t = 0 # number of times we have scrolled

    # loop the scrolling until cannot scroll anymore
    while True:
        print(t)
        # scroll down to bottom of whatever is currently loaded
        driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', scrolling_element)
        t = t+1

        # wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # check if more scrolling required
        new_height = driver.execute_script('return arguments[0].scrollHeight', scrolling_element)
        print(new_height)
        if new_height == last_height:
            break
        last_height = new_height

    stop =timeit.default_timer()

    print('Time taken: ', stop - start)

In [366]:
from selenium.webdriver.common.by import By

def extract_text(html, section, class_, find_span=False, multi_elements=False, multi_elements_type=None, aria_label=False):
    elements = html.find_all(section, class_=class_)
    #print(elements)
    if find_span:
        text = elements[1]
        text = re.sub(r'[^가-힣\s]', '', text.text)
        return text.strip()
    if multi_elements:
        text_list = []
        if multi_elements_type=='time':
            for e in elements[::2]:
                text_list.append(e.text.strip())
        elif multi_elements_type=='days':
            for e in elements[:int(len(elements)/2)]:
                text_list.append(e.text.strip())
        elif aria_label:
            for e in elements:
                text_list.append(e['aria-label'])
        else:
            for e in elements:
                #print(e)
                text_list.append(e.text.strip())
        return text_list
    else:
        return elements[0].text.strip()

In [392]:
def get_data(driver):
    """
    this function get main text, score, name
    """
    print('get data...')
    more_elements = driver.find_element(By.XPATH, "//button[@class='w8nwRe kyuRq']")
    more_elements.click()

    html = BeautifulSoup(driver.page_source, 'html.parser')
    elements = html.find_all('div', class_='jftiEf')
    reviews_data = []
    stars_data = []
    for data in elements:
        review = data.find('span', class_='wiI7pd')
        stars = data.find('span', class_='kvMYJc')
        review = review.text.strip() if review else None
        reviews_data.append(review)
        stars_data.append(stars['aria-label'])
    return reviews_data, stars_data

In [300]:
def click_element(driver, xpath):
    element = driver.find_element('xpath', xpath)
    element.click()
    time.sleep(2.0)

In [324]:
def get_opening_times(html):
    opening_times = extract_text(html, 'td', class_='mxowUb', multi_elements=True, multi_elements_type='time')
    days_of_week = extract_text(html, 'td', class_='ylH6lf', multi_elements=True, multi_elements_type='days')
    return list(zip(days_of_week, opening_times))

In [343]:
def back(driver, xpath, click_num):
    for i in range(click_num):
        click_element(driver, xpath)
        time.sleep(3.0)

In [371]:
url = 'https://www.google.com/maps/@33.5907877,130.3814759,14z/data=!3m1!4b1!4m3!11m2!2sVJhNXQIN1FHaiTf11xl6IV8gEwL_VQ!3e3?entry=ttu'

In [393]:
from itertools import cycle

data = {'place_jp': [],
        'address': [],
        'opening_times': [],
        'reviews_text': [],
        'reviews_stars': []}

# start driver
driver = webdriver.Chrome()
start = timeit.default_timer()
driver.get(url)
time.sleep(2.0)

errors = 0

for i in range(1, 4):
    print(i)
    try:
        if ((i-1)//20) > 0:
            load_listings(i)
        click_element(driver, xpath=f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[{str(i)}]')
        # get place and address info
        html = BeautifulSoup(driver.page_source, 'html.parser')
        data['place_jp'].append(extract_text(html, 'h2', class_='bwoZTb'))
        data['address'].append(extract_text(html, 'div', class_='rogA2c'))

        # get opening times for each place
        try:
            click_element(driver, xpath='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[5]/button/div[1]')
            html = BeautifulSoup(driver.page_source, 'html.parser')
            opening_times = get_opening_times(html)
            data['opening_times'].append(opening_times)
            back(driver, xpath='//*[@id="omnibox-singlebox"]/div/div[1]/button/span', click_num=1)
        except Exception:
            try:
                days_of_week = extract_text(html, 'td', class_='ylH6lf', multi_elements=True)
                opening_times = extract_text(html, 'li', class_='G8aQO', multi_elements=True)
                data['opening_times'].append([(x, ' '.join(opening_times[:2])) for x in days_of_week])
            except Exception:
                data['opening_times'].append(None)

        # get reviews for each place
        click_element(driver, xpath='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]')
        scroll_down()
        reviews_data, stars_data = get_data(driver)
        data['reviews_text'].append(reviews_data)
        data['reviews_stars'].append(stars_data)

    except NoSuchElementException:
        errors += 1
        print('running NoSuchElementException...sleeping 5 sec')
        time.sleep(2.0)
        back(driver, xpath='//*[@id="omnibox-singlebox"]/div/div[1]/button/span', click_num=1)
        if ((i-1)//20) > 0:
            load_listings(i)
        click_element(driver, xpath=f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[{str(i)}]')
        print('end of exception')

    back(driver, xpath='//*[@id="omnibox-singlebox"]/div/div[1]/button/span', click_num=2)

stop = timeit.default_timer()

print('time taken: ', stop - start)
print('errors: ', errors)

1
6824
0
8359
1
15060
2
21389
3
27601
4
33371
5
38036
6
41874
7
46274
8
48918
9
51556
10
59952
11
67938
12
74656
13
81788
14
88559
15
94893
16
100878
17
106184
18
110105
19
113856
20
117486
21
122322
22
126776
23
130352
24
133770
25
137753
26
141656
27
144287
28
147894
29
150595
30
152578
31
155268
32
157198
33
159161
34
161295
35
163076
36
165211
37
166581
38
167951
39
171096
40
173059
41
174429
42
175966
43
177783
44
179153
45
179524
46
179524
Time taken:  95.8613165880015
get data...
2
6469
0
12458
1
18154
2
23311
3
29424
4
35246
5
40932
6
45795
7
51635
8
57212
9
62570
10
66695
11
71226
12
74243
13
78731
14
82217
15
84861
16
88182
17
92203
18
98444
19
105101
20
111463
21
117495
22
123452
23
129451
24
135339
25
141401
26
147503
27
153912
28
159646
29
165472
30
170604
31
175904
32
180705
33
184306
34
188666
35
193866
36
199400
37
204036
38
207820
39
211890
40
216000
41
219816
42
223394
43
226971
44
230904
45
234587
46
238448
47
238448
Time taken:  98.39441383400117
get data...
3
6572


In [397]:
reviews_df = pd.DataFrame(data)
reviews_df.head(10)

,place_jp,address,opening_times,reviews_text,reviews_stars
0,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",[도쿄에서 이마한에 가지못해서 이번여행에 가장 기대한 이마한 스키야키입니다.\n굉장...,"[별표 5개, 별표 5개, 별표 4개, 별표 5개, 별표 5개, 별표 5개, 별표 ..."
1,田中田式海鮮食堂 魚忠(うおちゅう),"1 Chome-18-26 Imaizumi, Chuo Ward, Fukuoka, 81...","[(수요일, 휴무일 오전 11:30~오후 9:00), (목요일, 휴무일 오전 11:...",[한국과 가격은 비슷한데 확실히 회 퀄리티와 다양성은 확실히 더 좋은거 같아요!! ...,"[별표 5개, 별표 5개, 별표 5개, 별표 5개, 별표 4개, 별표 5개, 별표 ..."
2,やきにくのバクロ 博多店,"1 Chome-1-9-202 Sumiyoshi, Hakata Ward, Fukuok...","[(수요일, 오전 11:30~오후 2:30 오후 5:00~10:00), (목요일, ...",[정육왕님 유튜브 보고 방문 했습니다. 구글로 7시 예약 후 방문 했고 저희가 방문...,"[별표 5개, 별표 5개, 별표 5개, 별표 5개, 별표 5개, 별표 5개, 별표 ..."


In [398]:
df_expanded = reviews_df.explode(['reviews_text', 'reviews_stars'])
df_expanded.reset_index(drop=True, inplace=True)  # Reset index
df_expanded.head(10)

,place_jp,address,opening_times,reviews_text,reviews_stars
0,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",도쿄에서 이마한에 가지못해서 이번여행에 가장 기대한 이마한 스키야키입니다.\n굉장히...,별표 5개
1,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",모녀여행 마지막날 점심에 들렸습니다.\n미리 예약하고 갔고 음식점 입장할 때부터 퇴...,별표 5개
2,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",분위기와 직원분들의 친절함은 최상입니다. 고기가 부들부들 하니 맛 또한 좋습니다. ...,별표 4개
3,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",워크인으로 방문했더니 풀북이었어요. 큐알코드로 웹페이지 예약해서 식사했습니다. 정성...,별표 5개
4,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",예약을 미리 했었습니다.\n이토록 극진하게 정성스러운 대접을 받는것이 좋으면서도 처...,별표 5개
5,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",고급진 분위기에 가성비좋은 스키야끼코스. 점심특선이 정말 혜자입니다.\n후쿠오카 올...,별표 5개
6,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",여행의 마지막으로 가격은 부담되지만\n점심 일정으로 오픈런했습니다.\n평일 목요일이...,별표 4개
7,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",점심 인당 8470엔 코스 예약해서 먹었어요 테이블당 1명이 맡아서 서빙을 해주므로...,별표 5개
8,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",2023년 3월 19일 방문.\n전반적으로 매우 만족스러운 식사였음.\n한 명이 붙...,별표 5개
9,人形町今半 JR博多シティ店,"일본 〒812-0012 Fukuoka, Hakata Ward, Hakataekich...","[(수요일, 오전 11:00~오후 2:30오후 5:00~8:30), (목요일, 오전...",솔직히 저렴한 가격은 아니지만\n전혀 돈이 아깝지않은 최고의 서비스와 맛!\n마지막...,별표 5개
